The Martingale betting strategy is one where the gambler doubles their bet after each loss in order to cancel out previous losses and be left with a profit equal to the original stake. Proof:

a = original bet  
(2^0)a + (2^1)a + (2^2)a + (2^3)a + ... + (2^(n-1))a = (2^n)a - (2^0)a  
(2^0)a + (2^0)a + (2^1)a + (2^2)a + (2^3)a + ... + (2^(n-1))a = (2^n)a  
LHS = (2^1)a + (2^1)a + (2^2)a + ... + (2^(n-1))a  
...  
 = (2^(n-2))a + (2^(n-2))a + (2^(n-1))a  
 = (2^(n-1))a + (2^(n-1))a = RHS


In reality, this strategy is very risky as the gambler could go bust before winning, resulting in a huge loss that offsets all previous profits. In this notebook, I will run a series of experiments to analyse the risk/benefit of this strategy as applied to the casino game roulette.

In the below code, we will assume that if the gambler reaches a point where their remaining cash is less than the amount required to keep the strategy running (ie, twice the previous bet), they will go "all out" and bet their last remaining cash.

In [2]:
import numpy as np
import sys

PROB_OF_WIN = 18/37 #(18R, 18B, 1x0)

def spin():
    return np.random.rand() < PROB_OF_WIN #returns 1 or 0

def play(min_bet, max_cash, rounds, display_results=True): #rounds=0 to keep going until bust
    
    if max_cash < min_bet:
        sys.exit("Available cash insufficient to start the game")
        
    total_bets = []
    remaining_cash = max_cash
    game_over = False
    rspin = 0 #initialise
    
    if rounds != 0:
    
        while game_over == False:
            for i in range(rounds):
                bets = []
                stop_signal = False
                while stop_signal == False:
                    if len(bets) == 0:
                        bet = min_bet
                    else:
                        bet = min(bets[-1]*2, remaining_cash)
                    
                    rspin = spin()
                    
                    if rspin == 0:
                        bets.append(bet)
                        remaining_cash -= bet
                        
                        if remaining_cash == 0:
                            total_bets.append(np.sum(bets))
                            if display_results == True:
                                print(f"You went bust after {i+1} rounds")
                            game_over = True
                            stop_signal = True
                            break

                    if rspin == 1:
                        bets.append(bet)
                        total_bets.append(np.sum(bets))
                        remaining_cash += bet
                        stop_signal = True
                        if i == rounds-1:
                            game_over = True
                            
                if game_over == True:
                    break
        
        profits = remaining_cash - max_cash
        if display_results == True:
            if profits < 0:
                print(f"started with {max_cash} and ended up with {remaining_cash} leaving a loss of {profits}")
            else:
                print(f"started with {max_cash} and ended up with {remaining_cash} leaving a profit of {profits}")
            print("highest total bet:", np.max(total_bets))
        return profits

    
    
    #when rounds set to 0, keep going until bust
    else:
        rounds = 0
        while game_over == False:
            rounds += 1
            bets = []
            stop_signal = False

            while stop_signal == False:
                if len(bets) == 0:
                    bet = min_bet
                else:
                    bet = min(bets[-1]*2, remaining_cash)

                rspin = spin()

                if rspin == 0:
                    bets.append(bet)
                    remaining_cash -= bet

                    if remaining_cash == 0:
                        total_bets.append(np.sum(bets))
                        if display_results == True:
                            print(f"You went bust after {rounds} rounds")
                        game_over = True
                        stop_signal = True
                        break

                if rspin == 1:
                    bets.append(bet)
                    total_bets.append(np.sum(bets))
                    remaining_cash += bet
                    stop_signal = True

            if game_over == True:
                break
        if display_results == True:
            print("highest total bet:", np.max(total_bets))
        return rounds

Below see 3 runs of min_bet = 5, max_cash = 100, rounds = 10

In [3]:
for a in range(3):
    print(f"run: {a+1}")
    play(min_bet = 5, max_cash = 100, rounds = 10)
    print("")

run: 1
You went bust after 8 rounds
started with 100 and ended up with 0 leaving a loss of -100
highest total bet: 135

run: 2
started with 100 and ended up with 115 leaving a profit of 15
highest total bet: 120

run: 3
started with 100 and ended up with 150 leaving a profit of 50
highest total bet: 75



Let's run an experiment to find out how profitable this strategy is in the long run.

In [4]:
experiment = play(min_bet = 5, max_cash = 100, rounds = 10)
experiment

started with 100 and ended up with 150 leaving a profit of 50
highest total bet: 35


50

Let's define an experiment that can average results to give us a reliable average for a chosen scenario.

In [5]:
def experiment(n, max_cash, rounds, display = True):
    TPL = []
    for i in range(n):

        PL = []
        for j in range(1000):
            PL.append(play(5, max_cash, rounds, display_results = False))
        TPL.append(sum(PL))
        
    if display == True:
        if sum(TPL) < 0:
           print(f"Average losses = {round(sum(TPL)/len(TPL),2)}")
        else:
           print(f"Average profits = {round(sum(TPL)/len(TPL),2)}")
    return round(sum(TPL)/len(TPL),2)

Let's average 100 scenarios of playing with max 500 for 5 rounds, 1000 times:

In [6]:
experiment(100,500,5)

Average losses = -4638.1


-4638.1

Let's try again, setting the rounds to 1 (least risk of going bust)

In [7]:
experiment(100,500,1)

Average losses = -1115.85


-1115.85

Below I tried repeating the experiment increasing the max cash by a factor of 10 until reaching 1,000,000 and averaged 100 times for each one. Intuitively, it would seem that the more cash you start with, the less chance of going bust (with a million you'd have to lose at least 20 spins in a row). But it seems from the results that this is disproportionally outweighed by the increase in potential loss. 

The profits after each round if you do not go bust, do not exceed the minimum bet, thus after 100 rounds the most you can make is 500 (with a 5 minimum), yet the amount you can lose is equal to the full amount of cash you have. Thus, even if the strategy makes you win money with far greater frequency than you lose it, it only takes going bust once to completely wipe out all your winnings.

In [8]:
max_cash = [1000,10000,100000,1000000]
for cash in max_cash:
    a = []
    for i in range(100):
        a.append(experiment(100,cash,1,False))
    print(round(sum(a)/len(a),2))

-1170.29
-1697.02
-2419.39
-2202.17


Finally, let's see how many rounds it does take to go bust if we start with 1 million, by setting rounds to 0 and averaging over 100 results.

In [9]:
b = []
for i in range(100):
    b.append(play(min_bet = 5, max_cash = 1000000, rounds = 0, display_results=False))
print(f"On average it takes {round(sum(b)/len(b),0)} rounds to go bust")

On average it takes 271924.0 rounds to go bust


Now let's take a more realistic example. Let's see what happens if a gambler enters the casino with $10,000, committing to using the strategy for an entire day. As the winning odds are just below 50/50, on average it should take just over 2 spins per round. 

Proof:  

E(rounds) = 1 x (18/37) + 2 x (18/37) x (19/37) + 3 x (18/37) x (19/37)^2 + ...  
= (18/37)*(37/19) [(19/37) + 2(19/37)^2 + 3(19/37)^3 + ....]  
The term in brackets can be expressed as an infinite sum of na^n where a is (19/37).  
= ~(18/37)*(37/19)*2.169753 = 2.0555..

Googling tells me casinos average 45 spins/hour, so assuming the gambler is playing 12 hours straight, that's 540 spins in total, 270 rounds.

In [10]:
b = 0
a = 19/37
for i in range(1000000):
    b += i*a**i
print(b)
c = b*(18/37)*(37/19)
print(c)

2.1697530864197527
2.0555555555555554


In [11]:
a = 0
for i in range(10000):
    a += play(5, 10000, 270, 0) 
print(f"average profit/loss: {a/10000}")

average profit/loss: -450.8175


We average a loss...

Conclusion:
The martingale strategy is not a reliable method to make money from gambling. Essentially you concentrate the risk from losing more frequently into one massive potential loss with lower probability. Frequent low rewards with a high chance balanced with infrequent high losses with low chance. 

**The House always wins**